In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import random
import urllib.parse
from bs4 import BeautifulSoup
import json
import urllib.parse
from selenium.common.exceptions import ElementNotInteractableException, StaleElementReferenceException,ElementClickInterceptedException


In [2]:
with open('large_links.txt',encoding='utf-8') as f:
    links_raw = f.readlines()
print(len(links_raw))
links = list(set(links_raw))
print(len((links)))

2821
2763


In [3]:
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    count = 0
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for page to load
        time.sleep(random.uniform(1.5, 2.5))

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            count += 1
            if count == 30:
                break
        else:
            last_height = new_height
            count = 0

        # Simulate human-like scrolling by scrolling in smaller increments
        #for i in range(3):
         #   driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
          #  time.sleep(random.uniform(0.5, 1))
def extract_text(url,file_name):
    print('starting******************wait until closing**********')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)    
    driver.get(url)
    scroll_to_bottom(driver)
    print('scrolling completed')
    # Find all clickable "மேலும் காண்க" elements
    wait = WebDriverWait(driver, 10)
    elements = wait.until(ec.presence_of_all_elements_located((By.XPATH, "//*[contains(text(), 'மேலும் காண்க' or 'more')]")))
    clickable_elements = []
    for element in elements:
        if element.is_displayed() and element.is_enabled():
            clickable_elements.append(element)

    # Click all the clickable "மேலும் காண்க" elements
    for element in clickable_elements:
        driver.execute_script("arguments[0].click();", element)
    print('clicking complete')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # Find all elements with class 'qu-mb--tiny'
    small_classes = soup.find_all(class_='qu-mb--tiny')
    x = 0
    # Loop over the small_classes and find the adjacent 'spacing_log_answer_content' class
    for small_class in small_classes:
        adjacent_class = small_class.find_next(class_='spacing_log_answer_content')
        if adjacent_class:     
            mydict = {}
            question = small_class.text.strip()
            answer = adjacent_class.text.strip()
            if len(question)>30:
                x = x+1
                mydict["question"] = question
                mydict["answer"] = answer
                # Open the JSON file in append mode
                with open(file_name, "a",encoding='utf-8') as f:
                    f.write(str(mydict))
                    f.write('\n')
    print('closing***********************************************')
    driver.quit()    

# for quora topics

In [ ]:
from tqdm import tqdm
n=0
for i in tqdm(links):
    n+=1
    encoded_text = i.split('/')[-1]
    decoded_text = urllib.parse.unquote(encoded_text)[9:].replace(' ','_')
    print(decoded_text)
    try:
        file_name = '/home/aki/GPT/tamil_llm/output_topics/{}_{:04d}.txt'.format(decoded_text,n)
        extract_text(i,file_name)
    except:
        pass   

  0%|                                                   | 0/394 [00:00<?, ?it/s]

தமிழ்

starting******************wait until closing**********
scrolling completed
clicking complete


  0%|                                       | 1/394 [04:55<32:18:41, 295.98s/it]

closing***********************************************
புவியியல்_தொடர்பான_குறுங்கட்டுரைகள்

starting******************wait until closing**********


# for quora search and extract

In [4]:
def extract_text_search_query(url,filename):
    ls = []
    print("starting scraping of {}***************************".format(url))
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)    
    driver.get(url)
    print('started scrolling********************')
    scroll_to_bottom(driver)
    print('Completed scrolling********************')
    print('Started clicking********************')
    click_element = driver.find_elements(By.CSS_SELECTOR,".qt_read_more")
    for element in click_element:        
        driver.execute_script("arguments[0].click();", element)
    print('Finished clicking********************')
    question_element = driver.find_elements(By.CSS_SELECTOR,".qu-pb--tiny")
    print('extracting text **************************************')
    for element in question_element:
        q = element.find_elements(By.CSS_SELECTOR,".puppeteer_test_question_title")
        a = element.find_elements(By.CSS_SELECTOR,".puppeteer_test_answer_content")
        q_text = ''
        a_text = ''
        for i in q:
            q_text = q_text + i.text
        for j in a:
            a_text = a_text + j.text
        if a_text:     
            mydict = {}
            if len(q_text)>20:                
                    mydict["question"] = q_text
                    mydict["answer"] = a_text
                    # Open the JSON file in append mode
                    ls.append(mydict)
        print('Finished extracting text***********************')
    with open(filename, 'w',encoding='utf-8') as f:
        json.dump(ls, f, ensure_ascii=False, indent=4)
    
    

In [ ]:
from tqdm import tqdm
import os 
scraped_links= os.listdir('/home/aki/GPT/tamil_llm/output/')
scraped_links_processed=[]
for link in scraped_links:
    scraped_links_processed.append(link[:-11])
n=0
for i in tqdm(links):
    n+=1    
    encoded_text = i.split('/')[-1]
    decoded_text = urllib.parse.unquote(encoded_text)[9:].replace(' ','_')
    if decoded_text not in scraped_links_processed:
        print(decoded_text)
        try:
            file_name = '/home/aki/GPT/tamil_llm/output/{}_{:04d}.json'.format(decoded_text,n)
            extract_text_search_query(i,file_name)
        except:
            pass  

  0%|                                                  | 0/2763 [00:00<?, ?it/s]

சின்ன_மரங்கொத்தி

starting scraping of https://ta.quora.com/search?q=சின்ன மரங்கொத்தி
***************************
started scrolling********************
Completed scrolling********************
Started clicking********************
Finished clicking********************
extracting text **************************************
Finished extracting text***********************
Finished extracting text***********************


  0%|                                       | 1/2763 [01:06<51:06:30, 66.61s/it]

Finished extracting text***********************
Finished extracting text***********************
சின்ன_வல்லூறு

starting scraping of https://ta.quora.com/search?q=சின்ன வல்லூறு
***************************
started scrolling********************
Completed scrolling********************
Started clicking********************
Finished clicking********************
extracting text **************************************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************


  0%|                                       | 2/2763 [02:14<51:32:41, 67.21s/it]

...

Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text****************

Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text****************

Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text***********************
Finished extracting text****************

  1%|▌                                 | 41/2763 [1:01:37<126:49:17, 167.73s/it]

Finished extracting text***********************
Finished extracting text***********************
அகிலாண்டம்

starting scraping of https://ta.quora.com/search?q=அகிலாண்டம்
***************************
started scrolling********************


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import random
import urllib.parse

url_raw = "https://ta.quora.com/search?q=தமிழ் இலக்கணம்"
url = urllib.parse.unquote(url_raw)
# Load the web page
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome()
driver.get(url)
scroll_to_bottom(driver)
print('scrolling completed')
# Find all clickable "மேலும் காண்க" elements
wait = WebDriverWait(driver, 10)
elements = wait.until(ec.presence_of_all_elements_located((By.XPATH, "//*[contains(text(), '(மேலும் காண்க)' or '(more)')]")))
clickable_elements = []
for element in elements:
    if element.is_displayed() and element.is_enabled():
        clickable_elements.append(element)

# Click all the clickable "மேலும் காண்க" elements
for element in clickable_elements:
    driver.execute_script("arguments[0].click();", element)
print('clicking complete')

html = driver.page_source

from bs4 import BeautifulSoup
import json
soup = BeautifulSoup(html, 'html.parser')

soup = BeautifulSoup(html, 'html.parser')

# Find all elements with class 'qu-mb--tiny'
small_classes = soup.find_all(class_='qu-mb--tiny')
x = 0
# Loop over the small_classes and find the adjacent 'spacing_log_answer_content' class
for small_class in small_classes:
    adjacent_class = small_class.find_next(class_='spacing_log_answer_content')
    if adjacent_class:     
        mydict = {}
        question = small_class.text.strip()
        answer = adjacent_class.text.strip()
        if len(question)>30:
            x = x+1
            mydict["question"] = question
            mydict["answer"] = answer
            # Open the JSON file in append mode
            with open("qna_quora.txt", "a",encoding='utf-8') as f:
                f.write(str(mydict))          

#finding next sibling
from bs4 import BeautifulSoup
# Create a BeautifulSoup object
soup = BeautifulSoup(html, 'html.parser')
question_elements = soup.find_all(class_='qu-mb--tiny')
# Find all 'qu-mb--tiny' elements and iterate over each of them
count = 0
for question in question_elements:
    # Find the next sibling element with class 'spacing_log_answer_content'
    #answer = question.find_next_sibling(class_='spacing_log_answer_content')  
    # Extract the text from both elements
    if question:
        #question_text = question.text(strip=True)
        #answer_text = answer.text(strip=True)
        # Print the question and answer
        count+=1
        print(count)
        print(question.get_text().strip())
        #print('Answer:', answer_text)
        print('------------------------')


In [14]:
import os 
scraped_links= os.listdir('/home/aki/GPT/tamil_llm/output/')
scraped_links_processed=[]
for link in scraped_links:
    scraped_links_processed.append(link[:-11])
scraped_links_processed

['தொகைச்சொல்_அ_கூட்டுச்சொல்',
 'வெள்ளைக்கண்_வைரி',
 'விதைப்பந்து',
 'அளிக்கி',
 'ஆனைக்கு_ஒரு_காலம்_பூனைக்கு_ஒரு_காலம்',
 'பொரி_வல்லூறு',
 'ஐந்தில்_வளையாதது_ஐம்பதில்_வளையாது',
 'வெள்ளை_வாலாட்டி',
 'இராமாயணம்',
 'பிங்_யாவோ_பண்டைய_நகரம்_ஷாங்சி_மாகாணம்',
 'அலை_மோதும்போதே_தலை_முழுகு',
 'விசைப்பலகை',
 'மாரியம்மன்_தாலாட்டு',
 'உடலும்_பாலுறவும்',
 'நீலவால்_பஞ்சுருட்டான்',
 'இடம்_கொடுத்தால்_மடம்_பிடுங்குவான்',
 'இசைவூசல்',
 'கரிவிற்ற_பணம்_கறுப்பாய்_இருக்குமா',
 'திருஞானசம்பந்தமூர்த்தி_நாயனார்',
 'பெனின்',
 'வஞ்சகம்_வாழ்வைக்_கெடுக்கும்',
 'போர்த்துகல்',
 'கல்லாதார்_செல்வத்திலும்_கற்றார்_வறுமை_நலம்',
 'ஆயுத_எழுத்து',
 'நான்மணிக்கடிகை',
 'உயிரிவேதியியல்',
 'போசோன்கள்',
 'சிறு_பக்கி',
 'கலகம்_பிறந்தால்_நியாயம்_பிறக்கும்',
 'சொறிதல்',
 'நல்_இணக்க_மல்லது_அல்லற்_படுத்தும்',
 'எச்மியாட்சின்னின்_தேவாலயங்கள்_மற்றும்_சிற்றாலயங்களும்_ஸ்வாட்ஸ்நாட்ஸ்_தொல்லியல்_களமும்',
 'குயில்',
 'மிதித்தாரை_கடியாத_பாம்பு_உண்டோ',
 'மச்சமுனி',
 'கண்டம்_கருப்பை_வாய்',
 'கள்ள_மனம்_துள்ளும்',
 'ஓட்டுதல்',
 'வேற்றுமை_தமிழ்_இலக்க

In [7]:
scraped_link[0][:-11].replace('_',' ')

'தொகைச்சொல் அ கூட்டுச்சொல்'